In [4]:
# Keras implementation of Encoder-Decoder architecture with Attention to determine 
# the merchant name from description (name + purpose)
# Author: Karthik D

%matplotlib inline
import pandas as pd
import numpy as np
from numpy import argmax
from numpy import array
from numpy import array_equal
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from attention_decoder import AttentionDecoder
from keras.layers import Dropout
from sklearn.model_selection import train_test_split

In [13]:
# Some variables
num_epochs=100
PAD = '<PAD>'
seq_len = 27

# Load Data

In [7]:
raw_part2 = pd.read_csv('challenge_data/challenge_data_part2.csv', sep=";")
raw_part2.columns = [name.replace(" ", "_") for name in list(raw_part2.columns)]
raw_part2.columns

Index(['transaction_id', 'cents', 'booking_date', 'name', 'purpose',
       'booking_text', 'merchant'],
      dtype='object')

# Some data cleaning

In [8]:
raw = pd.DataFrame(raw_part2.name.str.replace(r'[0-9!@$%^&*()+_,\-:./]', ""), columns=['name']).fillna('')
raw['purpose'] = raw_part2.purpose.str.replace(r'[0-9!@$%^&*()+_,\-:/.]', "").fillna('')
raw['booking_text'] = raw_part2.booking_text.str.replace(r'[0-9!@$%^&*()+_,\-:/.]', "").fillna('')
raw['merchant'] = raw_part2.merchant.str.replace(r'[0-9!@$%^&*()+_,\-:/.]', "")
raw['x'] = raw.name.str.lower() + " " + raw.purpose.str.lower()

dev = raw.dropna(how='any')
len(dev)
test = raw[raw.merchant.isnull()].drop("merchant", axis=1)
len(test)

100

# Train Test Split

In [9]:
x_train, x_test, y_train, y_test = train_test_split(dev.drop('merchant', axis=1), dev.merchant, test_size=0.2, random_state=0)
print("Train Size:", x_train.shape)
print("Val size:", x_test.shape)

Train Size: (80, 4)
Val size: (20, 4)


# Define utility functions

In [16]:
# Func to make vocab from the data frame
def make_vocab(df):
    tokens = df.x + " " + df.merchant.fillna('')
    vocab = set()
    tokens.str.split().apply(vocab.update)
    while '' in vocab:
        vocab.remove('')
    vocab.add(PAD)
    print("Vocab size:", len(vocab))
    return vocab


# Make word to index dict
def make_w2i(vocab):
    w2i = {}
    i=0
    for word in vocab:
        if not word in w2i:
            w2i[word] = i
            i+=1
    return w2i


# Wrapper function to make a torch variable
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)


# Function to make word padding
def padding(data, max_sent_len, pad_token):
    pad_len = max(0, max_sent_len - len(data))
    data += [pad_token] * pad_len
    data = data[:max_sent_len]
    return data


# Wrapper to make a tensor from a list/np
def make_tensor(data, w2i):
    #seq_len = max(data.str.split().apply(len))
    ret_data = [padding([w2i[word] for word in row.split()], seq_len, w2i[PAD]) for row in data]
    return to_var(torch.LongTensor(ret_data))


# Wrapper to make a tensor from a list/np
def make_tensor2(data, w2i):
    #seq_len = max(data.str.split().apply(len))
    ret_data = [padding([w2i[word] for word in row.split()], seq_len, w2i[PAD]) for row in data]
    return np.array(ret_data)


# one hot encode sequence
def one_hot_encode(sequence):
    encoding = list()
    for value in sequence:
        vector = [0 for _ in range(len(vocab))]
        vector[value] = 1
        encoding.append(vector)
    return array(encoding)


# a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]


# Get pairs of predictors and target
def get_pair(n_in, n_out=None):
    X = one_hot_encode(n_in)
    # reshape as 3D
    X = X.reshape((1, X.shape[0], X.shape[1]))
    if n_out is not None:
        y = one_hot_encode(n_out)
        # reshape as 3D
        y = y.reshape((1, y.shape[0], y.shape[1]))
    else:
        y = None
    return X,y


#Get a list of keys from dictionary which has value that matches with any value in given list of values
def getKeysByValues(listOfValues):
    listOfKeys = list()
    listOfItems = w2i.items()
    for item  in listOfItems:
        if item[1] in listOfValues and item[0] != PAD:
            listOfKeys.append(item[0])      
    return  listOfKeys

In [18]:
vocab = make_vocab(raw)
w2i = make_w2i(vocab)
train_x = make_tensor2(x_train.x.str.strip(), w2i)
train_y = make_tensor2(y_train.str.strip(), w2i)

test_x = make_tensor2(x_test.x.str.strip(), w2i)
test_y = make_tensor2(y_test.str.strip(), w2i)

print("Train x tensor shape:", train_x.shape)
print("Train y tensor shape:", train_y.shape)

print("Test x tensor shape:", test_x.shape)
print("Test y tensor shape:", test_y.shape)

Vocab size: 670
Train x tensor shape: (80, 27)
Train y tensor shape: (80, 27)
Test x tensor shape: (20, 27)
Test y tensor shape: (20, 27)


# Keras Encoder-Decoder with Attention

In [25]:
# configure problem
n_features = len(vocab) 
# define model
model = Sequential()
model.add(LSTM(150,  input_shape=(train_x.shape[1], n_features)))
#model.add(Dropout(rate=0.2))
model.add(RepeatVector(train_y.shape[1]))
model.add(AttentionDecoder(150, n_features))
#model.add(Dropout(rate=0.2))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

# Train Model

In [26]:
# train LSTM
for epoch in range(num_epochs):
    for i in range(train_x.shape[0]):
        X, y = get_pair(train_x[i], train_y[i])
        # fit model for one epoch on this sequence
        model.fit(X, y, epochs=1, verbose=2)

Epoch 1/1
 - 2s - loss: 6.5050 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 6.4312 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 6.3414 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 6.2566 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 6.1458 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 5.9358 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 5.7219 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 5.3936 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 4.5056 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 3.7479 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 3.0810 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 1.8585 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 1.2037 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 1.1778 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.6409 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 1.5336 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.2788 - acc: 0.9630
Epoch 1/1
 - 0s - loss: 0.8643 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 1.6609 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 1.3129 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 1.7407 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.8624 - acc: 0.8889
Epoch 

Epoch 1/1
 - 0s - loss: 0.5364 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.5750 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.7376 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.3898 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.3549 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 0.3835 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.4610 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.4441 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.3645 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 0.3077 - acc: 0.9630
Epoch 1/1
 - 0s - loss: 0.5101 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.6706 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.3299 - acc: 0.9630
Epoch 1/1
 - 0s - loss: 0.4820 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 0.5096 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.6661 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.3939 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 0.3678 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.2066 - acc: 0.9630
Epoch 1/1
 - 0s - loss: 0.7071 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.6967 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.2979 - acc: 0.8889
Epoch 1/1


 - 0s - loss: 0.6144 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.4361 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.5607 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.6426 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 15.3004 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 0.4839 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.2830 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 0.4021 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.2799 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 0.5762 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.5485 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.3791 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.5633 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.5582 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.5094 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.7132 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.5100 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.5096 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.4476 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.4469 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.4877 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.4767 - acc: 0.8148
Epoch 1/1
 - 0s

 - 0s - loss: 0.4110 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 0.3886 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 0.4543 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.5641 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.3104 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.3909 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.3799 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.3633 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.3307 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.3384 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.3220 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.4315 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.4624 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.5366 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.3457 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.3894 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.4875 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.9198 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.8165 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.8814 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.2619 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 0.2580 - acc: 0.9259
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 0.2457 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 0.7478 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.2270 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 0.5070 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.2073 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 0.4288 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.5376 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.5166 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.4502 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.6175 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.4962 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.9986 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.4050 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.5242 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.5252 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.5937 - acc: 0.8148
Epoch 1/1
 - 0s - loss: 0.2492 - acc: 0.9259
Epoch 1/1
 - 0s - loss: 0.3527 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.3631 - acc: 0.8889
Epoch 1/1
 - 0s - loss: 0.4721 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.4593 - acc: 0.8519
Epoch 1/1
 - 0s - loss: 0.3514 - acc: 0.9259
Epoch 1/1


...

 - 0s - loss: 8.8946e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0040 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0016 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 5.1955e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 6.9299e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 6.8431e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 5.6264e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.4803e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 5.7318e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0057 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0039 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 5.2667e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 8.4533e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 5.2176e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 5.3214e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 3.3523e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0019 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0033 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 5.1647e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.4928e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.5506e-04 - acc: 1.0000
Epoch

Epoch 1/1
 - 0s - loss: 2.8809e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0017 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0028 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 4.3218e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.3349e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.3842e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0013 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0035 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0040 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0036 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0011 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.3394e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.3518e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0022 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0034 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0018 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0014 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 4.5719e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.3230e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.3333e-04 - acc: 1.0000
Epoch 1/1
 - 0s - l

Epoch 1/1
 - 0s - loss: 0.0028 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0016 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0012 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 3.9776e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.1881e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.1968e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0012 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.1601e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.2452e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 3.8892e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0011 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 5.0862e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0028 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0028 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0013 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0020 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0013 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.2072e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.2016e-04 - acc: 1.0000
Epoch 1/1
 - 0s - l

Epoch 1/1
 - 0s - loss: 0.0023 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0024 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0012 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0018 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0011 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.0867e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.0815e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 5.8718e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.1262e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0038 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.1167e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0023 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.0445e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0011 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 4.4562e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0033 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0010 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0010 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 4.5750e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0025 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 3.4818e-04 - acc: 1.0000
Epoch 1/1
 - 0s - l

Epoch 1/1
 - 0s - loss: 9.3424e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 3.9910e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0028 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 9.2449e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 9.2626e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 4.1026e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0022 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 3.0626e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 7.7732e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0016 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 5.0095e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 9.4384e-05 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 2.7177e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 3.0059e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 7.4767e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 8.2579e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 5.6625e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 2.9598e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0014 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0018 - acc: 1.

# Evaluate on Training set

In [29]:
# evaluate LSTM
correct =  0
for i in range(train_x.shape[0]):
    X,y = get_pair(train_x[i], train_y[i])
    yhat = model.predict(X, verbose=0)
    print("Actual:", getKeysByValues(one_hot_decode(y[0])))
    print("Pred:", getKeysByValues(one_hot_decode(yhat[0])))
    if array_equal(one_hot_decode(y[0]), one_hot_decode(yhat[0])):
        correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(train_x.shape[0])*100.0))

Actual: ['AG', 'Allianz', 'Deutschland']
Pred: ['AG', 'Allianz', 'Deutschland']
Actual: ['congstar', 'GmbH']
Pred: ['congstar', 'GmbH']
Actual: ['Bettenlager', 'Daenisches']
Pred: ['Bettenlager', 'Daenisches']
Actual: ['Mauritz', 'GesmbH', 'Hennes', 'M', 'H']
Pred: ['Mauritz', 'GesmbH', 'Hennes', 'M', 'H']
Actual: ['Beitragsservice', 'ZDF', 'ARD', 'Deutschlandradio']
Pred: ['Beitragsservice', 'ZDF', 'ARD', 'Deutschlandradio']
Actual: ['Mauritz', 'GesmbH', 'Hennes', 'M', 'H']
Pred: ['Mauritz', 'GesmbH', 'Hennes', 'M', 'H']
Actual: ['Amazon', 'Services', 'Europe', 'Sà', 'rl']
Pred: ['Amazon', 'Services', 'Europe', 'Sà', 'rl']
Actual: ['Amazon', 'Services', 'Europe', 'Sà', 'rl']
Pred: ['Amazon', 'Services', 'Europe', 'Sà', 'rl']
Actual: ['Zalando', 'SE']
Pred: ['Zalando', 'SE']
Actual: ['Amazon', 'Services', 'Europe', 'Sà', 'rl']
Pred: ['Amazon', 'Services', 'Europe', 'Sà', 'rl']
Actual: ['AG', 'CTS', 'EVENTIM', 'KGaA', 'Co']
Pred: ['AG', 'CTS', 'EVENTIM', 'KGaA', 'Co']
Actual: ['Amazon',

# Evaluate on Test set

In [30]:
# evaluate LSTM
correct = 0
for i in range(test_x.shape[0]):
    X,y = get_pair(test_x[i], test_y[i])
    yhat = model.predict(X, verbose=0)
    print("Actual:", getKeysByValues(one_hot_decode(y[0])))
    print("Pred:", getKeysByValues(one_hot_decode(yhat[0])))
    if array_equal(one_hot_decode(y[0]), one_hot_decode(yhat[0])):
        correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(test_x.shape[0])*100.0))

Actual: ['Einkauf', 'GmbH', 'ALDI', 'OHG', 'Co']
Pred: ['Einkauf', 'GmbH', 'ALDI', 'OHG', 'Co']
Actual: ['Stiftung', 'Lidl', 'KG', 'Co']
Pred: ['Stiftung', 'Lidl', 'KG', 'Co']
Actual: ['Telefónica', 'GmbH', 'Germany', 'OHG', 'Co']
Pred: ['AG', 'IKEA', 'KG', 'ZENTRALE', 'Co']
Actual: ['Amazon', 'Services', 'Europe', 'Sà', 'rl']
Pred: ['Amazon', 'Services', 'Europe', 'Sà', 'rl']
Actual: ['Amazon', 'Services', 'Europe', 'Sà', 'rl']
Pred: ['Amazon', 'Services', 'Europe', 'Sà', 'rl']
Actual: ['AG', 'HDI', 'Deutschland']
Pred: ['AG', 'IKEA', 'KG', 'Deutschland', 'Co']
Actual: ['Stiftung', 'Lidl', 'KG', 'Co']
Pred: ['Stiftung', 'Lidl', 'KG', 'Co']
Actual: ['KG', 'Kaufland', 'GmbH', 'Dienstleistung', 'Co']
Pred: ['KG', 'Kaufland', 'GmbH', 'Dienstleistung', 'Co']
Actual: ['mobilcomdebitel', 'GmbH']
Pred: ['mobilcomdebitel', 'GmbH']
Actual: ['AG', 'Post', 'Deutsche']
Pred: ['AG', 'Post', 'Deutsche']
Actual: ['Einkauf', 'GmbH', 'ALDI', 'OHG', 'Co']
Pred: ['Einkauf', 'GmbH', 'ALDI', 'OHG', 'Co']
A

# Inferencing

In [31]:
test_x = make_tensor2(test.x.str.strip(), w2i)
pred =[]
# evaluate LSTM on test
for i in range(test_x.shape[0]):
    X, _ = get_pair(test_x[i])
    yhat = model.predict(X, verbose=0)
    pred.append(getKeysByValues(one_hot_decode(yhat[0])))
    #print("Pred:", getKeysByValues(one_hot_decode(yhat[0])))
    
test = test.drop('x', axis=1).drop('booking_text', axis=1)
test['predicted'] = pred
test.to_csv("predictions.csv")